In [117]:
import os
import numpy as np
import pandas as pd
from models import *   
import load_data
import nan_imputation
import helpers

In [119]:
# Reload isdead.py
import importlib
importlib.reload(load_data)
importlib.reload(nan_imputation)
importlib.reload(helpers)

<module 'helpers' from '/Users/louistschanz/Documents/EPFL-Cours/MA1/ML/Project-2/ML-Project-2/helpers.py'>

## Step 0 : Load the Data

In [120]:
repo_root = find_repo_root()
repo_root

# load data
# Define the path to the Lifespan folder

#for now test with only control
data_path = os.path.join(repo_root, 'Data/Lifespan')

In [ ]:
worms = load_data.load_lifespan(data_path)
#worms.pop("worm_1_companyDrug", None)  # The second argument avoids KeyError if the key doesn't exist

Loading file: /Users/louistschanz/Documents/EPFL-Cours/MA1/ML/Project-2/ML-Project-2/Data/Lifespan/companyDrug/speeds_and_coordinates_20241016_24_updated.csv
File loaded: /Users/louistschanz/Documents/EPFL-Cours/MA1/ML/Project-2/ML-Project-2/Data/Lifespan/companyDrug/speeds_and_coordinates_20241016_24_updated.csv
Loading file: /Users/louistschanz/Documents/EPFL-Cours/MA1/ML/Project-2/ML-Project-2/Data/Lifespan/companyDrug/speeds_and_coordinates_20241016_3_updated.csv
File loaded: /Users/louistschanz/Documents/EPFL-Cours/MA1/ML/Project-2/ML-Project-2/Data/Lifespan/companyDrug/speeds_and_coordinates_20241016_3_updated.csv
Loading file: /Users/louistschanz/Documents/EPFL-Cours/MA1/ML/Project-2/ML-Project-2/Data/Lifespan/companyDrug/speeds_and_coordinates_20241016_6_updated.csv
File loaded: /Users/louistschanz/Documents/EPFL-Cours/MA1/ML/Project-2/ML-Project-2/Data/Lifespan/companyDrug/speeds_and_coordinates_20241016_6_updated.csv
Loading file: /Users/louistschanz/Documents/EPFL-Cours/MA1/

In [122]:
helpers.print_fdict_summary(worms)  

# need to calibrate : 
#companyDrug :
# 1 : 53995
# 2 : 75594
# 3 : 75594
# 4 : 75594
# 5 : 75594
# 6 : 64795
# 21 : 75594
# 22 : 75594
# 23 : 75594
# 24 : 75594
# 25 : 75594
# 26 : 75594
# 
#  control :
# 11 : 75594
# 12 : 75594
# 13 : 75594
# 14 : 75594 
# 15 : 64795
# 16 : 75594
# 31 : 53996
# 32 : 75594
# 33 : 53996
# 34 : 53996
# 35 : 75594
# 36 : 75594


Summary of Loaded Worm Data:
Worm: worm_1
  Shape: (6, 75593)
----------------------------------------
Worm: worm_2
  Shape: (6, 75593)
----------------------------------------
Worm: worm_3
  Shape: (6, 64794)
----------------------------------------
Worm: worm_4
  Shape: (6, 75593)
----------------------------------------
Worm: worm_5
  Shape: (6, 75593)
----------------------------------------
Worm: worm_6
  Shape: (6, 75593)
----------------------------------------
Worm: worm_7
  Shape: (6, 75593)
----------------------------------------
Worm: worm_8
  Shape: (6, 75593)
----------------------------------------
Worm: worm_9
  Shape: (6, 75593)
----------------------------------------
Worm: worm_10
  Shape: (6, 53995)
----------------------------------------
Worm: worm_11
  Shape: (6, 75593)
----------------------------------------
Worm: worm_12
  Shape: (6, 75593)
----------------------------------------
Worm: worm_13
  Shape: (6, 75593)
----------------------------------------
Worm

## Step 1 : NaN imputation
> impute only on X and Y columns since only where there are NaN

In [ ]:
for name, lifespan_array in worms.items(): 
    print(f"Processing {name}")
    lifespan_arrayxy = lifespan_array[2:4,:]  # Extract columns for X and Y
    missing_sequences = nan_imputation.count_successive_missing(lifespan_arrayxy)
    for start, end, length in missing_sequences:
        print(f"  Missing sequence starts at column {start}, ends at column {end - 1}, length: {length}")

Processing worm_1
  Missing sequence starts at column 4691, ends at column 4709, length: 19
  Missing sequence starts at column 7386, ends at column 7444, length: 59
  Missing sequence starts at column 7449, ends at column 7467, length: 19
  Missing sequence starts at column 9899, ends at column 9908, length: 10
  Missing sequence starts at column 9972, ends at column 9994, length: 23
  Missing sequence starts at column 11698, ends at column 11704, length: 7
  Missing sequence starts at column 11709, ends at column 11749, length: 41
  Missing sequence starts at column 11766, ends at column 11772, length: 7
  Missing sequence starts at column 11811, ends at column 11827, length: 17
  Missing sequence starts at column 11836, ends at column 11857, length: 22
  Missing sequence starts at column 14592, ends at column 14602, length: 11
  Missing sequence starts at column 14617, ends at column 14629, length: 13
  Missing sequence starts at column 14829, ends at column 14830, length: 2
Process

In [124]:
print(lifespan_arrayxy)
print(f"Missing sequences for {name}: {missing_sequences}")

[[444.85492228 444.25789474 444.         ... 283.29149798 282.996
  282.55378486]
 [135.31606218 134.99473684 134.8994709  ... 181.52631579 181.48
  181.38247012]]
Missing sequences for worm_24: [(2699, 2852, 153), (6622, 6637, 15), (6696, 6729, 33), (6900, 6929, 29), (7063, 7078, 15), (7082, 7095, 13), (7174, 7200, 26), (8405, 8419, 14), (8476, 8481, 5), (8999, 9236, 237), (9860, 9900, 40), (11855, 11884, 29), (12598, 12620, 22), (12777, 12790, 13), (12889, 12941, 52), (12996, 13104, 108), (13174, 13228, 54), (13412, 13543, 131), (15600, 16199, 599), (19208, 19221, 13), (19238, 19539, 301), (19798, 19829, 31), (19861, 19931, 70), (19966, 20624, 658)]


In [125]:
# Rows to check for missing values (2:4 in zero-based indexing)
rows_to_check = slice(2, 4)  # Rows 2 and 3 not row 4

# Apply cut_array to each worm in the dataset
cut_nan_dict = {name: nan_imputation.cut_array(array, rows_to_check) for name, array in worms.items()}

In [126]:
# Print the shape of the filtered arrays
for name, item in cut_nan_dict.items():
    print(f'{name} : {item.shape}')

worm_1 : (6, 75343)
worm_2 : (6, 74855)
worm_3 : (6, 64534)
worm_4 : (6, 74307)
worm_5 : (6, 75395)
worm_6 : (6, 75593)
worm_7 : (6, 74496)
worm_8 : (6, 72844)
worm_9 : (6, 73638)
worm_10 : (6, 53811)
worm_11 : (6, 74689)
worm_12 : (6, 73516)
worm_13 : (6, 74414)
worm_14 : (6, 53396)
worm_15 : (6, 52787)
worm_16 : (6, 74062)
worm_17 : (6, 72595)
worm_18 : (6, 64759)
worm_19 : (6, 70217)
worm_20 : (6, 72216)
worm_21 : (6, 71822)
worm_22 : (6, 52217)
worm_23 : (6, 75159)
worm_24 : (6, 72932)


# Step 2 : Splitting the Data
> Split the worms in train worms and test worms

In [127]:
train_worms, test_worms = load_data.split_worms(worms, test_size=0.2)

print(f"Training Worms: {list(train_worms.keys())}")
print(f"Testing Worms: {list(test_worms.keys())}")

Split 24 worms into 19 training and 5 testing worms.
Training Worms: ['worm_16', 'worm_6', 'worm_19', 'worm_24', 'worm_14', 'worm_11', 'worm_18', 'worm_12', 'worm_15', 'worm_17', 'worm_2', 'worm_13', 'worm_22', 'worm_7', 'worm_10', 'worm_3', 'worm_23', 'worm_5', 'worm_20']
Testing Worms: ['worm_8', 'worm_9', 'worm_1', 'worm_4', 'worm_21']


# Step 3 : Figure out when do the worms die
>When we find out on which frame he dies, drop the frames after his death

In [128]:
import isdead
importlib.reload(isdead)

<module 'isdead' from '/Users/louistschanz/Documents/EPFL-Cours/MA1/ML/Project-2/ML-Project-2/isdead.py'>

In [ ]:
# Iterate through each worm in the dataset
for worm_name, worm_data in worms.items():
    print(f"Processing {worm_name}...")

    # Transpose worm_data for DataFrame creation
    df_worm = pd.DataFrame(
        worm_data.T,  # Transpose the array
        columns=['Frame', 'Speed', 'X', 'Y', 'Changed Pixels', 'Category']
    )

    # Use the estimate_dying_time function to find the dying frame
    result = estimate_dying_time(df_worm, movement_threshold)
    if result[0] is None:
        print(f"  {worm_name}: No inactivity detected. Retaining all data.")
        processed_worms[worm_name] = worm_data
        continue

Processing worm_1...
Processing worm_2...
Processing worm_3...
Processing worm_4...
Processing worm_5...
Processing worm_6...
Processing worm_7...
Processing worm_8...
Processing worm_9...
Processing worm_10...
Processing worm_11...
Processing worm_12...
Processing worm_13...
Processing worm_14...
Processing worm_15...
Processing worm_16...
Processing worm_17...
Processing worm_18...
Processing worm_19...
Processing worm_20...
Processing worm_21...
Processing worm_22...
Processing worm_23...
Processing worm_24...


In [ ]:
df_worm